In [2]:
import pandas as pd
import cobra
from cobra.flux_analysis import pfba
import json
import matplotlib.pyplot as plt
import openpyxl
from cobra import Model, Reaction, Metabolite
import numpy as np
from cobra.flux_analysis.variability import flux_variability_analysis
from cobra.sampling import sample
from statsmodels.stats.multitest import multipletests
from scipy import stats
from cameo import models
import os
#import plotly.express as px


In [3]:
os.system("pwd")

/Raid/raid6/database/wuliu.notebook/wuliu/cobra


0

In [5]:
cobra_config = cobra.Configuration()
cobra_config.solver = "glpk"
m = cobra.io.read_sbml_model('iCN1361/12H_biomass.xml')

In [6]:
m.solver

TypeError: 'Model' object is not callable

In [8]:
m.summary()

/home/wuliu/raid6/conda/envs/jupyterLab/lib/python3.9/site-packages/cobra/core/metabolite.py:103: UserWarning: 41.22 is not an integer (in formula C41.22H81.86O21.79N11.71)
  warn(
/home/wuliu/raid6/conda/envs/jupyterLab/lib/python3.9/site-packages/cobra/core/metabolite.py:103: UserWarning: 81.86 is not an integer (in formula C41.22H81.86O21.79N11.71)
  warn(
/home/wuliu/raid6/conda/envs/jupyterLab/lib/python3.9/site-packages/cobra/core/metabolite.py:103: UserWarning: 21.79 is not an integer (in formula C41.22H81.86O21.79N11.71)
  warn(
/home/wuliu/raid6/conda/envs/jupyterLab/lib/python3.9/site-packages/cobra/core/metabolite.py:103: UserWarning: 11.71 is not an integer (in formula C41.22H81.86O21.79N11.71)
  warn(


Metabolite,Reaction,Flux,C-Number,C-Flux
M_AMMONIUM,EX_AMMONIUM_e,2.704,0,0.00%
M_BETA_D_FRUCTOSE_e,EX_BETA_D_FRUCTOSE_e,2.6,6,100.00%
M_FE2,EX_FE2_e,0.001267,0,0.00%
M_OXYGEN_MOLECULE,EX_OXYGEN_MOLECULE_e,4.589,0,0.00%
M_Pi,EX_Pi_e,0.2253,0,0.00%
M_SULFATE,EX_SULFATE_e,0.04197,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
M_BIOMASS,EX_BIOMASS_e,-0.2595,41.22,68.48%
M_CARBON_DIOXIDE,EX_CARBON_DIOXIDE_e,-4.905,1.00,31.40%
M_CH33ADO,EX_CH33ADO_e,-0.001844,10.00,0.12%


In [55]:
#set fructose flux
m.reactions.R_FRUPTS.bounds = (0, 0) # previous study found the ABC is the active reaction [ref]
m.reactions.EX_BETA_D_FRUCTOSE_e.bounds = (0, 0)  
#m.reactions.EX_PALMITATE_e.bounds = (-1, -1) #0.3885
m.reactions.EX_OXYGEN_MOLECULE_e.bounds=(-1000,1000)
m.reactions.EX_CARBON_DIOXIDE_e.bounds = (-1000,1000)
#m.reactions.EX_OXYGEN_MOLECULE_e.bounds = (-7.5, -7.5)   
#m.reactions.R_Biomass.bounds = (0.022847, 0.022847)
#m.reactions.EX_PHA_12H_e.bounds = (2.1587, 2.1587)
#m.reactions.EX_AMMONIUM_e.bounds = (-0.2004, -0.2004)
#m.reactions.EX_OXYGEN_MOLECULE_e.bounds = (-5.1153, -5.1153)
#m.reactions.EX_CARBON_DIOXIDE_e.bounds = (1.8198, 1.8198)
#R_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN
#M_D_RIBULOSE_15_P2
#m.reactions.EX_CARBON_DIOXIDE_e.bounds = 
#m.reactions.R_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN.bounds = (-0.597, -0.597)
m.objective = 'R_Biomass'
m.reactions.EX_PALMITATE_e.bounds = (-1, 0)

In [56]:
solution = m.optimize()

In [ ]:
m.summary()

#control CBB and O2,detect oil usage

In [ ]:
import warnings
warnings.filterwarnings("ignore")
m.reactions.EX_CARBON_DIOXIDE_e.bounds = (-10, 10)

df = pd.DataFrame([[0,0,0,0]],columns = (['CBB', 'Biomass', 'O2','Oil']))
m.reactions.EX_PALMITATE_e.bounds = (-2, 0)

for i in np.arange(0,2,0.1):
    m.reactions.R_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN.bounds = (-i, -i)
    for j in np.arange(3,15,0.1):
        m.reactions.EX_OXYGEN_MOLECULE_e.bounds=(-j,-j)
        for x in np.arange(0,2,0.1):
            m.reactions.EX_PALMITATE_e.bounds = (-x, -x)
            Oil=x
            CBB=i
            O2=j
            solution = m.optimize()
            Biomass=solution.fluxes["R_Biomass"]
            #Oil = solution.fluxes["EX_PALMITATE_e"]
        #O2=solution.fluxes["EX_OXYGEN_MOLECULE_e"]
            df1 = pd.DataFrame(  [[ CBB , Biomass , O2,Oil] ] , columns = (['CBB','Biomass', 'O2',"Oil"]) )
            df  = pd.concat([df,df1]) 
            
#file="biomass.O2.CBB" + str(i) + '.' + str(j) +".csv"
#file="biomass.O2.CBB.oil.1" + str(i) + '.' + str(j) +".csv"
file="CBB.O2.oil.csv"
df.to_csv(file)

In [5]:
m.optimize

<bound method Model.optimize of <Model CnecH16 at 0x7f08389f5d60>>

In [1]:
a=1.334

In [2]:
a

1.334

In [6]:
print( "terst" + str(a))

terst1.334


In [6]:
m.reactions.EX_OXYGEN_MOLECULE_e.bounds=(-7,-7)
m.reactions.R_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN.bounds = (0, 0)
m.reactions.EX_PALMITATE_e.bounds = (-100, 0)
solution = m.optimize()


In [10]:
m.reactions.get_by_id("R_Biomass")

Reaction identifier,R_Biomass
Name,Biomass
Memory address,0x07f5341467a90
Stoichiometry,0.2508 M_ARG + 0.20295 M_ASN + 30.2922 M_ATP + 0.0538687561 M_CARBS + 3.4592723045332e-05 M_CLPN_C12 + 0.0020921778 M_CLPN_C14 + 3.0099e-05 M_CLPN_C15 + 0.0012405406 M_CLPN_C16 + 0.0008561755... 0.2508 L-arginine + 0.20295 L-asparagine + 30.2922 ATP + 0.0538687561 + 3.4592723045332e-05 + 0.0020921778 + 3.0099e-05 + 0.0012405406 + 0.0008561755 + 2.77001160042504e-05 + 3.2503e-05 +...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [ ]:
solution.fluxes["R_Biomass"]
m.summary()

In [8]:
solution.fluxes["EX_PALMITATE_e"]

-0.9537804867330539

#control CBB,Oil,detect O2 usage ,no achohole product

In [10]:
# control CBB, oil ,detect O2 and biomass ,as liuyedao did
import warnings
m.reactions.EX_CARBON_DIOXIDE_e.bounds = (-10,10)
m.reactions.EX_ACET_e.bounds = (0,0)
warnings.filterwarnings("ignore")
df = pd.DataFrame([[0,0,0,0]],columns = (['CBB', 'Biomass', 'O2','Oil']))
#m.reactions.EX_PALMITATE_e.bounds = (-1, -1)

for i in np.arange(0,2,0.01):
    m.reactions.EX_PALMITATE_e.bounds = (-i, -i)
    for j in np.arange(0,2,0.01):
        m.reactions.R_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN.bounds = (-j, -j)
        Oil=i
        CBB=j
        solution = m.optimize()
        Biomass=solution.fluxes["R_Biomass"]
        #Oil = solution.fluxes["EX_PALMITATE_e"]
        O2=solution.fluxes["EX_OXYGEN_MOLECULE_e"]
        df1 = pd.DataFrame(  [[ CBB , Biomass , O2,Oil] ] , columns = (['CBB','Biomass', 'O2',"Oil"]) )
        df  = pd.concat([df,df1]) 
#file="biomass.O2.CBB" + str(i) + '.' + str(j) +".csv"
file="aaa.control.oil.CBB.findO2.biomass.2.2.csv"
df.to_csv(file)

In [ ]:






df2 = df[df > 0].dropna() 
from scipy.stats import zscore
df3=df2.apply(zscore)


In [ ]:
m.reactions.EX_PALMITATE_e.bounds = (-10, 0)
m.reactions.EX_OXYGEN_MOLECULE_e.bounds=(-7,-7)
m.reactions.R_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN.bounds = (-0, -0)
solution1 = m.optimize()
m.objective = 'R_Biomass'
m.metabolites.M_OXYGEN_MOLECULE.summary()
outfile="flux.cbb0.csv"
solution1.fluxes.to_csv(outfile)

In [ ]:
solution1.fluxes["EX_PALMITATE_e"]


In [ ]:
m.reactions.EX_OXYGEN_MOLECULE_e.bounds=(-7,-7)
m.reactions.EX_PALMITATE_e.bounds = (-10, 0)
m.reactions.R_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN.bounds = (-0.6, -0.6)
solution2 = m.optimize()
m.objective = 'R_Biomass'
m.metabolites.M_PALMITATE.summary()
outfile="flux.cbb0.6.csv"
solution2.fluxes.to_csv(outfile)

In [ ]:
solution2.fluxes["EX_PALMITATE_e"]

In [ ]:
#import plotly.express as px
#df = px.data.gapminder().query("country=='Canada'")
#fig = px.line(df, x="CBB", y="Biomass", title='')
#px.density_heatmap(df, x="O2", y="CBB", marginal_x="histogram", marginal_y="histogram",text_auto=True)
import plotly.graph_objects as go

In [1]:
import plotly.express as px
#fig = px.line_3d(df2, x="O2", y="CBB", z="Biomass",height =1000 ,width =1000 )
fig= px.scatter_3d(df2, x="CBB", y="Biomass", z="O2",height =1000 ,width =1000,color="Biomass",size="O2", opacity=0.7 )
#fig.add_vline(0.6)
fig.show()

#px.scatter_3d()

NameError: name 'df2' is not defined

In [ ]:
fig.write_html("CBB.O2.biomass.html")

In [ ]:
z_data = pd.read_csv('test.csv.txt')
fig = go.Figure(data=[go.Surface(z=df)])
fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))
fig.update_layout(title='O2 CBB Biomass', autosize=False,
                  scene_camera_eye=dict(x=1.87, y=0.88, z=-0.64),
                  width=900, height=900,
                  margin=dict(l=65, r=50, b=65, t=90)
)

fig.show()

In [ ]:
fig.show(height=10, )

In [ ]:
#plt.plot(mass.keys(),mass.values(),marker='.',color="red", linestyle="dashed")
#plt.axis([0, 6, 0, 20])
#plt.show 

In [ ]:
m.reactions.EX_BETA_D_FRUCTOSE_e.bounds = (-16/6, -16/6) 
m.reactions.EX_PALMITATE_e.bounds = (0, 0)
m.objective = 'R_Biomass'
solution = m.optimize()

In [ ]:
fmass={}
for i in np.arange(-2,0,0.001):
    m.reactions.R_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN.bounds = (i, i)
    solution = m.optimize()
    fmass[i]=solution.fluxes["R_Biomass"]

In [ ]:
data={"CBB":fmass.keys(),"Biomass":fmass.values()}
df2 = pd.DataFrame(data)
fig = px.line(df2, x="CBB", y="Biomass", title='')

In [ ]:
fig.show()

In [ ]:
cobra.io.save_json_model(m,"CBB_Palmitate-CBB-50.model.json")

In [ ]:
outfile="flux.50.csv"
solution.fluxes.to_csv(outfile)

In [ ]:
m.metabolites.M_OXYGEN_MOLECULE.summary()

In [ ]:
#m.metabolites.M_2_KETOGLUTARATE.summary()
#m.reactions.get_by_id("R_MALIC_NADP_RXN")
m.Biomass

In [9]:
m.metabolites.M_Biomass.summary()

/home/wuliu/raid6/conda/envs/jupyterLab/lib/python3.9/site-packages/cobra/core/dictlist.py:215: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if current_length is 0:
/home/wuliu/raid6/conda/envs/jupyterLab/lib/python3.9/site-packages/cobra/core/dictlist.py:215: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if current_length is 0:
/home/wuliu/raid6/conda/envs/jupyterLab/lib/python3.9/site-packages/cobra/core/dictlist.py:215: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if current_length is 0:
/home/wuliu/raid6/conda/envs/jupyterLab/lib/python3.9/site-packages/cobra/core/dictlist.py:215: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if current_length is 0:


AttributeError: DictList has no attribute or entry M_Biomass

In [ ]:
m.metabolites.M_FRUCTOSE_6P.summary()

In [ ]:
m.metabolites.M_FRUCTOSE_16_DIPHOSPHATE.summary()

In [ ]:
m.metabolites.M_2_KETO_3_DEOXY_6_P_GLUCONATE.summary()

In [ ]:
m.metabolites.M_D_SEDOHEPTULOSE_7_P.summary()

In [ ]:
m.metabolites.M_DIHYDROXY_ACETONE_PHOSPHATE.summary()

In [ ]:
m.metabolites.M_RIBOSE_5P.summary()

In [ ]:
m.metabolites.M_RIBULOSE_5P.summary()

In [ ]:
m.metabolites.M_D_RIBULOSE_15_P2.summary()

In [ ]:
m.metabolites.M_G3P.summary()

In [ ]:
m.metabolites.M_GAP.summary()

In [ ]:
m.metabolites.M_FRUCTOSE_16_DIPHOSPHATE.summary()

In [ ]:
m.metabolites.M_PHOSPHO_ENOL_PYRUVATE.summary()

In [ ]:
m.metabolites.M_FUM.summary()

In [ ]:
m.metabolites.M_CIT.summary()

In [ ]:
m.metabolites.M_THREO_DS_ISO_CITRATE.summary()

In [ ]:
m.metabolites.M_GLYOX.summary()

In [ ]:
m.metabolites.M_GLYOX.summary()

In [ ]:
m.metabolites.M_Ubiquinones.summary()


In [ ]:
m.metabolites.M_ATP.summary()

In [ ]:
m.metabolites.M_OXALACETIC_ACID.summary()

In [ ]:
m.metabolites.M_CARBON_DIOXIDE.summary()

In [ ]:
 m.metabolites.M_HCO3.summary()

In [ ]:
m.metabolites.M_ACETYL_COA.summary()

In [ ]:
m.metabolites.M_2_KETOGLUTARATE.summary()

In [ ]:
m.metabolites.M_PYRUVATE.summary() 

In [ ]:
m.metabolites.M_PHOSPHO_ENOL_PYRUVATE.summary() 

In [ ]:
m.metabolites.M_2_PG.summary() 

In [ ]:
m.metabolites.M_G3P.summary() 

In [ ]:
m.metabolites.M_DPG.summary() 

In [ ]:
m.metabolites.M_GAP.summary() 

In [ ]:
m.metabolites.M_NADH.summary() 

In [ ]:
m.metabolites.M_ATP.summary() 

In [ ]:
m.metabolites.M_OXYGEN_MOLECULE.summary() 

In [ ]:
m.metabolites.M_NADPH.summary()

In [ ]:
生成accoa和biomass时的nadh

In [ ]:
把NADH都用于cbb时，对biomass的影响

In [ ]:
accoa的flux最大时